Here we have to put together tatoeba, jpWaC and wikipedia into a uniform file.
The columns should be: sentence, (diff_level), source, (doc analysis)

In [1]:
import pandas as pd
import re
import spacy
import ginza
import re

In [2]:
nlp = spacy.load('ja_ginza_electra', enable='')
nlp.add_pipe('sentencizer')
tokenizer = nlp.tokenizer

/home/enrico_benedetti/anaconda3/envs/nlp_env/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'ja_ginza_electra' (5.1.3) was trained with spaCy v3.2.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
def expand_sentences(sentences):
    """There are rows that contain multiple sentences according to our criteria.
    So, to avoid problems, we expand them here. Note that here we lose match with the original dataset."""
    docs = list(nlp.pipe(sentences))
    exp_sentences = []
    for doc in docs:
        for sent in doc.sents:
            exp_sentences.append(sent.as_doc().text)
    return exp_sentences

In [66]:
# try and apply the isgood to tatoeba also
def is_good(sentence, token_limit=50, punct_ratio=0.2, numeral_ratio=0.2, tokenizer=None):
    """Basic surface level requirements for sentence inclusion. requires spacy.
    If a list of tokens is passed, the doc is retrieved.
    If a sentence is passed, it will be tokenized if also the spacy tokenizer is passed, otherwise it will raise exception..."""
    # based on english characters and urls
    # based on having at least 5 tokens [sangawa paper] and ending in punctuation, and ending with a ADJ, VERB, AUX.
    
    if isinstance(sentence, list):
        # retrieve the doc from a list of tokens
        sentence = sentence[0].doc
    elif isinstance(sentence, str):
        sentence = tokenizer(sentence)

    sentence_length = len(sentence) # in tokens!!
    if sentence_length < 5 or sentence_length > token_limit:
        # print('len')
        return False
    
    if (sentence[-1].pos_ != 'PUNCT') or (sentence[-2].pos_ not in ['AUX', 'ADJ', 'VERB']):
    # print('ending')
        return False
    # pattern = r'[a-zA-Z]|https?:\/\/\S+'
    # if re.search(pattern, sentence.text) is not None:
    #     return False
    
    # no more than 20% punctuation or numerals
    #punct_match = re.findall(r'[!\"#$%&\'()*+,-./:;<=>?@[\\\]^_``{|}~…]', sentence.text)
    #punct_count = len(punct_match)
    # we do not consider the last
    punct_count = sum([token.is_punct for token in sentence[0:-1]])
    if punct_count / sentence_length > punct_ratio:
        #print('too much punct',sentence_length, punct_count, punct_count / sentence_length)
        return False
    
    # num_match = re.findall(r'\d', sentence.text)
    # num_count = len(num_match)
    num_count = sum([token.is_digit for token in sentence]) # digit actually means a full token of digits
    if num_count / sentence_length > numeral_ratio:
        #print('too much num',sentence_length, punct_count, punct_count / sentence_length)
        return False
    # no text in other languages
    if re.search(r'.*[A-Za-z].*', sentence.text):
        # print('english')
        return False
    
    arabic_pattern = r'[\u0600-\u06FF\u0750-\u077F]+'
    if re.search(arabic_pattern, sentence.text):
        # print('arabic')
        return False

    russian_pattern = r'[А-Яа-яЁё]+'
    if re.search(russian_pattern, sentence.text):
        # print('arabic')
        return False


    return True

def last_filters(sentence):
    """str based filter to remove outliers from the final dataset"""
    arabic_pattern = r'[\u0600-\u06FF\u0750-\u077F]+'
    if re.search(arabic_pattern, sentence):
        # print('arabic')
        return False

    russian_pattern = r'[А-Яа-яЁё]+'
    if re.search(russian_pattern, sentence):
        # print('arabic')
        return False
    
    if len(sentence) < 5 or len(sentence) > 100: # in chars!!
        return False

    return True
        


In [5]:
out_file = '../corpus_all/corpus.csv'
jpwac_file = '../corpora_original/jpWaC/jpWaC.csv'
tatoeba_file = "../corpora_original/tatoeba/tatoeba.csv"
wikipedia_file = '../corpora_original/wikipedia/wikipedia.txt'

In [6]:
jpwac_df = pd.read_csv(jpwac_file, usecols=["sentence"])
print(len(jpwac_df))
jpwac_df = pd.DataFrame({'sentence' : expand_sentences(jpwac_df['sentence'])})
print(len(jpwac_df))
jpwac_df['source'] = 'jpwac'
jpwac_df['good'] = jpwac_df['sentence'].map(lambda s: is_good(s, token_limit=50, tokenizer=tokenizer))
jpwac_df

152689
152925


,sentence,source,good
0,藤井氏の著書の販売から、ここでしか買えない音声メールマガジンのコーナーもあります。,jpwac,True
1,朱肉のつけすぎは、他人に写し取られて悪用されかねませんので、注意。,jpwac,False
2,特にこれに関して習得したいと思われるテーマがあれば気軽にリクエスト下さい。,jpwac,True
3,それに伴い、各種問い合わせを受付開始いたします。,jpwac,True
4,英語とか韓国語、中国語での出版も希望しています。,jpwac,True
...,...,...,...
152920,新しい1年が今，始まろうとしています。,jpwac,True
152921,また、どんな言葉をかけたらいいのかも分からなかった。,jpwac,True
152922,私も今となってはわかる。,jpwac,True
152923,よくわかりますよ。,jpwac,False


In [29]:
print(jpwac_df.good.value_counts())

good
True     117960
False     34965
Name: count, dtype: int64


In [8]:
tatoeba_df = pd.read_csv(tatoeba_file, usecols=["sentence"])
# here we expand the sentences
print(len(tatoeba_df))
tatoeba_df = pd.DataFrame({'sentence' : expand_sentences(tatoeba_df['sentence'])})
print(len(tatoeba_df))
tatoeba_df['source'] = 'tatoeba'
tatoeba_df['good'] = tatoeba_df['sentence'].map(lambda s: is_good(s, token_limit=50, tokenizer=tokenizer))
tatoeba_df

239380
248522


,sentence,source,good
0,きみにちょっとしたものをもってきたよ。,tatoeba,False
1,何かしてみましょう。,tatoeba,True
2,私は眠らなければなりません。,tatoeba,True
3,何してるの？,tatoeba,False
4,今日は６月１８日で、ムーリエルの誕生日です！,tatoeba,True
...,...,...,...
248517,ワインをお願い。,tatoeba,False
248518,彼らは私の話を信じようとしなかった。,tatoeba,True
248519,ハロウィンの仮装だよ。,tatoeba,False
248520,トムの誕生日に、新しい自転車をプレゼントするんだ。,tatoeba,True


In [12]:
print(tatoeba_df.good.value_counts())

good
True     182743
False     65779
Name: count, dtype: int64


In [45]:
# matcher = spacy.matcher.Matcher(nlp.vocab)
# pattern = [
#     {"POS": {"in": ["VERB", "AUX", "ADJ"]}},
#     {"POS": "PART", "OP": "{,2}"},  # match at most 2 ending particles
#     {"POS": "PUNCT"} 
#     # no way to easily say end of sentence, but we know that we already split them.
# ]
# matcher.add("sentence_ending", [pattern])

# # doc.match

In [10]:
wikipedia_df = pd.read_csv(wikipedia_file, header=None, names=['sentence'])
wikipedia_df['source'] = 'wikipedia'
wikipedia_df['good'] = True # assume already analyzed

In [13]:
wikipedia_df

,sentence,source,good
0,アンパサンドの起源は1世紀の古ローマ筆記体にまでさかのぼることができる。,wikipedia,True
1,それに続く、流麗さを増した新ローマ筆記体では、さまざまな合字が極めて頻繁に使われるようになった。,wikipedia,True
2,その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れて...,wikipedia,True
3,1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアン...,wikipedia,True
4,アンパサンドのルーツはローマ時代にさかのぼるため、ラテンアルファベットを使用する多くの言語で...,wikipedia,True
...,...,...,...
12628015,1883年9月4日、道路、運河、港湾、鉱山に関する学校として設立された。,wikipedia,True
12628016,1901年8月17日、高等工業学校が追加された。,wikipedia,True
12628017,1975年にはムルシア大学に組み込まれてカルタヘナ工科学校となった。,wikipedia,True
12628018,1998年8月3日にムルシア大学から分離され、大学としてのカルタヘナ工科大学が開学した。,wikipedia,True


In [46]:
# then,
# 0 merge all of them
df = pd.concat([jpwac_df, tatoeba_df, wikipedia_df], ignore_index=True)
# 1 remove duplicates
df_no_dup = df
sent_no_dup = df['sentence'].drop_duplicates()
df_no_dup['sentence'] = sent_no_dup
df_no_dup = df_no_dup.dropna(axis='index', how='any')

# 1.5 apply isgood filter? yes but no discard
#df_all_good = df_no_dup[df_no_dup['good']]
# 2 reset index / shuffle??
#df_all_good = df_all_good.reset_index(drop=True)
# 3 save as corpus.csv
#df_all_good.to_csv(out_file, columns=['sentence', 'source'], index=False)

In [47]:
# alternate - save also the bad part... but bad sentences will still be used, I think.
df_all = df_no_dup.reset_index(drop=True)
filtered = df_all['sentence'].apply(last_filters)
df_all = df_all[filtered]
df_all = df_all.reset_index(drop=True)
df_all.to_csv(out_file, columns=['sentence', 'source', 'good'], index=False)

In [50]:
# its better to save only the sentences and put the indeces in readme or other file...
# we save 200MB
df_all.to_csv(out_file, columns=['sentence'], index=False)
first_unique_index = df_all[df_all['source'].duplicated() == False].index
df_all['source'].loc[first_unique_index].to_csv('sources.csv')

In [65]:
df_all

,sentence,source,good
0,藤井氏の著書の販売から、ここでしか買えない音声メールマガジンのコーナーもあります。,jpwac,True
1,朱肉のつけすぎは、他人に写し取られて悪用されかねませんので、注意。,jpwac,False
2,特にこれに関して習得したいと思われるテーマがあれば気軽にリクエスト下さい。,jpwac,True
3,それに伴い、各種問い合わせを受付開始いたします。,jpwac,True
4,英語とか韓国語、中国語での出版も希望しています。,jpwac,True
...,...,...,...
12704955,1883年9月4日、道路、運河、港湾、鉱山に関する学校として設立された。,wikipedia,True
12704956,1901年8月17日、高等工業学校が追加された。,wikipedia,True
12704957,1975年にはムルシア大学に組み込まれてカルタヘナ工科学校となった。,wikipedia,True
12704958,1998年8月3日にムルシア大学から分離され、大学としてのカルタヘナ工科大学が開学した。,wikipedia,True


In [62]:
df_all.isna().value_counts()

sentence  source  good 
False     False   False    12704960
Name: count, dtype: int64

In [64]:
read_test = pd.read_csv(out_file)
read_test

,sentence
0,藤井氏の著書の販売から、ここでしか買えない音声メールマガジンのコーナーもあります。
1,朱肉のつけすぎは、他人に写し取られて悪用されかねませんので、注意。
2,特にこれに関して習得したいと思われるテーマがあれば気軽にリクエスト下さい。
3,それに伴い、各種問い合わせを受付開始いたします。
4,英語とか韓国語、中国語での出版も希望しています。
...,...
12704955,1883年9月4日、道路、運河、港湾、鉱山に関する学校として設立された。
12704956,1901年8月17日、高等工業学校が追加された。
12704957,1975年にはムルシア大学に組み込まれてカルタヘナ工科学校となった。
12704958,1998年8月3日にムルシア大学から分離され、大学としてのカルタヘナ工科大学が開学した。
